Setup and Auth

In [2]:
from transformers import pipeline
from huggingface_hub import login
from dotenv import load_dotenv
import os

# Load token from .env
load_dotenv()
login(os.getenv("HF_API_KEY"))

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load Analysis Data

In [3]:
import pandas as pd
import sqlite3
from scipy.stats import linregress

conn = sqlite3.connect("../climate_data.db")
df = pd.read_sql_query("SELECT * FROM avg_temperatures_cleaned", conn)
conn.close()

# Choose one city as example
city = "Rome"
df_city = df[df["City"] == city]

slope, intercept, r, p, _ = linregress(df_city["Year"], df_city["AvgMaxTemp"])

text = (
    f"From 2014 to 2023, {city} experienced a warming trend. "
    f"The average maximum temperature increased from {df_city['AvgMaxTemp'].iloc[0]:.1f}°C "
    f"to {df_city['AvgMaxTemp'].iloc[-1]:.1f}°C. "
    f"The trend slope was {slope:.3f} with a p-value of {p:.4f}, indicating "
    f"{'statistical significance' if p < 0.05 else 'no significant trend'}."
)
print(text)


From 2014 to 2023, Rome experienced a warming trend. The average maximum temperature increased from 20.4°C to 21.7°C. The trend slope was 0.370 with a p-value of 0.0742, indicating no significant trend.


Summary

In [6]:
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

print("🔎 Summary:")
print(summary[0]["summary_text"])


Device set to use cpu


🔎 Summary:
 Zurich experienced a warming trend from 2014 to 2023 . The average max temp increased from 14.8°C to 15.9°C .


Summary for all cities

In [7]:
for city in df["City"].unique():
    df_city = df[df["City"] == city]
    slope, intercept, r, p, _ = linregress(df_city["Year"], df_city["AvgMaxTemp"])

    text = (
        f"From 2014 to 2023, {city} experienced a warming trend. "
        f"The average max temp increased from {df_city['AvgMaxTemp'].iloc[0]:.1f}°C "
        f"to {df_city['AvgMaxTemp'].iloc[-1]:.1f}°C. "
        f"Slope = {slope:.3f}, p-value = {p:.4f}. "
        f"This trend is {'statistically significant' if p < 0.05 else 'not statistically significant'}."
    )

    summary = summarizer(text, max_length=50, min_length=20, do_sample=False)
    print(f"\n📍 {city} Summary:\n{summary[0]['summary_text']}")



📍 Zurich Summary:
 Zurich experienced a warming trend from 2014 to 2023 . The average max temp increased from 14.8°C to 15.9°C .

📍 Rome Summary:
 Rome experienced a warming trend from 2014 to 2023 . The average max temp increased from 20.4°C to 21.7°C . Slope = 0.370, p-value = 0 .0742. Slope

📍 London Summary:
 From 2014 to 2023, London experienced a warming trend . The average max temp increased from 14.9°C to 15.4°C . Slope = 0.137, p-value = 0 .4403. Sl
